Inspiration: https://towardsdatascience.com/recurrent-neural-networks-rnns-3f06d7653a85

# Imports and globals

In [137]:
import numpy as np

# Basic functions

In [138]:
def sigmoid(x):
    return 1. / (1. + np.exp(-x))

def sigmoid_derivative(sigmoid_output):
    return sigmoid_output * (1 - sigmoid_output)

def softmax_naive(x):
    return np.exp(x) / np.sum(np.exp(x))

def softmax_stable(x):
    p = np.exp(x - np.max(x))
    return p / np.sum(p)

softmax = softmax_stable

softmax(np.array([range(3)]))

array([[0.09003057, 0.24472847, 0.66524096]])

In [257]:
class RNN:
    def __init__(self, input_size:int, hidden_size:int, output_size:int):
        np.random.seed(1234)
        self.Wxh = np.random.uniform(0, 1, (hidden_size, input_size))
        self.Whh = np.random.uniform(0, 1, (hidden_size, hidden_size))
        self.Why = np.random.uniform(0, 1, (output_size, hidden_size))
        self.bh = np.random.uniform(0, 1, (hidden_size, 1))
        self.by = np.random.uniform(0, 1, (output_size, 1))
        self.reset_history()

    def reset_history(self):
        h = np.zeros_like(self.bh)
        self.h_history = [h]
        self.y_history = []

    @property
    def seq_length(self):
        return len(self.y_history)

    @property
    def h(self):
        return self.h_history[-1]

    @h.setter
    def h(self, value):
        self.h_history.append(value)

    @property
    def y(self):
        return self.y_history[-1]

    @y.setter
    def y(self, value):
        self.y_history.append(value)

    def forward_one_step(self, x):
        z = np.dot(self.Wxh, x) + np.dot(self.Whh, self.h) + self.bh
        self.h = np.tanh(z)
        o = np.dot(self.Why, self.h) + self.by
        self.y = softmax(o)
        return self.y

    def forward(self, xs):
        res = []
        for x in xs:
            res.append(self.forward_one_step(x))
        return res

    def loss(self, targets):
        return sum(-np.log(np.take(self.y_history[t], targets)) for t in range(self.seq_length))

    def backward(self, xs, targets):
        dWxh = np.zeros_like(self.Wxh)
        dWhh = np.zeros_like(self.Whh)
        dWhy = np.zeros_like(self.Why)
        dbh = np.zeros_like(self.bh)
        dby = np.zeros_like(self.by)
        dparams = (dWxh, dWhh, dWhy, dbh, dby)
        dhnext = np.zeros_like(self.h)
        for t in reversed(range(self.seq_length)):
            dy = np.copy(self.y_history[t])
            dy[targets[0, t]] -= 1
            dWhy += np.dot(dy, self.h_history[t].T)
            dby += dby
            dh = np.dot(self.Why.T, dy) + dhnext
            dhrec = (1 - self.h_history[t] * self.h_history[t]) * dh
            dbh += dhrec
            dWxh += np.dot(dhrec, xs[t].T)
            dWhh += np.dot(dhrec, self.h_history[t - 1].T)
            dhnext = np.dot(self.Whh.T, dhrec)
        for dparam in dparams:
            np.clip(dparam, -5, 5, out=dparam)
        return dparams

    def update(self, dparams, learning_rate):
        dWxh, dWhh, dWhy, dbh, dby = dparams
        self.Wxh -= dWxh * learning_rate
        self.Whh -= dWhh * learning_rate
        self.Why -= dWhy * learning_rate
        self.bh -= dbh * learning_rate
        self.by -= dby * learning_rate

    def train(self, xs, targets, iters, learning_rate):
            for i in range(iters):
                self.reset_history()
                preds = self.forward(xs)
                loss = rnn.loss(targets)
                if i % 1000 == 0:
                    print('Predictions:', preds)
                    print('Loss:', loss)
                dparams = rnn.backward(xs, targets)
                self.update(dparams, learning_rate)

In [258]:
rnn = RNN(2, 4, 3)
xs = [np.array([[1], [3]]), np.array([[2], [-6]])]
targets = np.zeros((1, 3), dtype=np.int64)

rnn.train(xs, targets, 100000, 0.01)

      [2.61720531e-15],
       [1.46703782e-10]]), array([[9.99133581e-01],
       [6.80842560e-05],
       [7.98334502e-04]])]
Loss: [[0.00086679 0.00086679 0.00086679]]
Predictions: [array([[1.00000000e+00],
       [2.52991405e-15],
       [1.37970318e-10]]), array([[9.99248255e-01],
       [5.60561177e-05],
       [6.95688880e-04]])]
Loss: [[0.00075203 0.00075203 0.00075203]]
Predictions: [array([[1.00000000e+00],
       [2.45614706e-15],
       [1.30738038e-10]]), array([[9.99335807e-01],
       [4.73294197e-05],
       [6.16863125e-04]])]
Loss: [[0.00066441 0.00066441 0.00066441]]
Predictions: [array([[1.00000000e+00],
       [2.39251829e-15],
       [1.24612916e-10]]), array([[9.99404844e-01],
       [4.07486944e-05],
       [5.54407688e-04]])]
Loss: [[0.00059533 0.00059533 0.00059533]]
Predictions: [array([[1.00000000e+00],
       [2.33674489e-15],
       [1.19332969e-10]]), array([[9.99460678e-01],
       [3.56332369e-05],
       [5.03688846e-04]])]
Loss: [[0.00053947 0.0005394